**INVERTED MNIST IMAGING**

In [3]:
# Import necessary libraries.
import torch
import torch.nn as nn
import sys
import time
from typing import Union, List, Dict, Any, cast
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
batch_size=64

# Use the MODIFIED transform function with the lambda function to ensure images are being inverted.
transform = transforms.Compose([transforms.Resize(224),
                                transforms.ToTensor(),
                                transforms.Lambda(lambda x: 1 - x),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root="./data",train=True, transform=transform, download=True)
testset = torchvision.datasets.MNIST(root="./data",train=False, transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False)

# Define VGG structure.
def vgg_block(num_convs, in_channels, num_channels):
    layers=[]
    for i in range(num_convs):
        layers+=[nn.Conv2d(in_channels=in_channels, out_channels=num_channels, kernel_size=3, padding=1)]
        in_channels=num_channels
    layers +=[nn.BatchNorm2d(num_channels)]
    layers +=[nn.ReLU()]
    layers +=[nn.MaxPool2d(kernel_size=2, stride=2)]
    return nn.Sequential(*layers)

# Build the CNN.
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv_arch=((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
        layers=[]
        for (num_convs,in_channels,num_channels) in self.conv_arch:
            layers+=[vgg_block(num_convs,in_channels,num_channels)]
        self.features=nn.Sequential(*layers)
        self.dense1 = nn.Linear(512*7*7,4096)
        self.drop1 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(4096, 4096)
        self.drop2 = nn.Dropout(0.5)
        self.dense3 = nn.Linear(4096, 10)

    # Forward pass.
    def forward(self,x):
        x=self.features(x)
        x=x.view(-1,512*7*7)
        x=self.dense3(self.drop2(F.relu(self.dense2(self.drop1(F.relu(self.dense1(x)))))))
        return x

# Assess the accuracy of the function.
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if ('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

# Define the training structure for the network.
def train(train_iter, test_iter, net, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    collaps = 0
    simplex = torch.zeros(len(train_iter.dataset),10).to(device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for  X, y in tqdm(train_iter):
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1

100%|██████████| 9.91M/9.91M [00:00<00:00, 15.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 406kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.79MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.4MB/s]


In [4]:
# Train the model through 2 epochs.
lr = 0.003
num_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net1 = Net()
optimizer = torch.optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)
train(trainloader, testloader, net1, optimizer, device, num_epochs)

training on cuda


100%|██████████| 938/938 [08:42<00:00,  1.80it/s]


In [5]:
# Evaluate accuracy and print.
test_acc = evaluate_accuracy(testloader, net1)
print(test_acc)

0.9883


**UNMODIFIED MNIST IMAGING**

In [7]:
# Import necessary libraries.
import torch
import torch.nn as nn
import sys
import time
from typing import Union, List, Dict, Any, cast
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

batch_size=64

# Use the MODIFIED transform function WITHOUT LAMBDA FUNCTION.
transform = transforms.Compose([transforms.Resize(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root="./data",train=True, transform=transform, download=True)
testset = torchvision.datasets.MNIST(root="./data",train=False, transform=transform, download=True)
trainloader = torch.utils.data.DataLoader(dataset=trainset,batch_size=batch_size,shuffle=True)
testloader = torch.utils.data.DataLoader(dataset=testset,batch_size=batch_size,shuffle=False)

# Define VGG structure.
def vgg_block(num_convs, in_channels, num_channels):
    layers=[]
    for i in range(num_convs):
        layers+=[nn.Conv2d(in_channels=in_channels, out_channels=num_channels, kernel_size=3, padding=1)]
        in_channels=num_channels
    layers +=[nn.BatchNorm2d(num_channels)]
    layers +=[nn.ReLU()]
    layers +=[nn.MaxPool2d(kernel_size=2, stride=2)]
    return nn.Sequential(*layers)

# Builds the CNN.
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv_arch=((1,1,64),(1,64,128),(2,128,256),(2,256,512),(2,512,512))
        layers=[]
        for (num_convs,in_channels,num_channels) in self.conv_arch:
            layers+=[vgg_block(num_convs,in_channels,num_channels)]
        self.features=nn.Sequential(*layers)
        self.dense1 = nn.Linear(512*7*7,4096)
        self.drop1 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(4096, 4096)
        self.drop2 = nn.Dropout(0.5)
        self.dense3 = nn.Linear(4096, 10)

    def forward(self,x):
        x=self.features(x)
        x=x.view(-1,512*7*7)
        x=self.dense3(self.drop2(F.relu(self.dense2(self.drop1(F.relu(self.dense1(x)))))))
        return x

# Assess the accuracy of the function.
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        device = list(net.parameters())[0].device
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                net.eval()
                acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
                net.train()
            else:
                if ('is_training' in net.__code__.co_varnames):
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item()
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
            n += y.shape[0]
    return acc_sum / n

# Define the training structure for the network.
def train(train_iter, test_iter, net, optimizer, device, num_epochs):
    net = net.to(device)
    print("training on", device)
    loss = torch.nn.CrossEntropyLoss()
    batch_count = 0
    collaps = 0
    simplex = torch.zeros(len(train_iter.dataset),10).to(device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for  X, y in tqdm(train_iter):
            X = X.to(device)
            y = y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()

            train_l_sum += l.cpu().item()
            train_acc_sum += (y_hat.argmax(dim=1) == y).sum().cpu().item()
            n += y.shape[0]
            batch_count += 1

In [8]:
# Train the model through 2 epochs.
lr = 0.003
num_epochs = 2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

net1 = Net()
optimizer = torch.optim.SGD(net1.parameters(), lr=0.001, momentum=0.9)
train(trainloader, testloader, net1, optimizer, device, num_epochs)

training on cuda


100%|██████████| 938/938 [08:38<00:00,  1.81it/s]


In [9]:
# Evaluate accuracy and print.
test_acc = evaluate_accuracy(testloader, net1)
print(test_acc)

0.9897
